In [40]:
import xlwings as xw
import pandas as pd

In [41]:
df = ""
df_criterio = ""
df_bancada = ""

In [42]:
# Função para verificar se a emenda está na 'Bancada Impositiva - Consulta'
def checa_bancada_impositiva(emenda, df_bancada):
    
    # Verifica se a emenda está presente na coluna 'Emenda' da aba 'Bancada Impositiva - Consulta'
    return emenda in df_bancada['Emenda'].values

In [43]:
# Função para calcular a 'Decisão' de acordo com a lógica fornecida
def calcula_decisao(row, df_bancada):
    # Se a soma de 'Atendimento Setorial' não for zero
    if row['Soma Atendimento Setorial'] != 0:
        # Se a soma de 'Atendimento Setorial' for igual à soma de 'Valor Solicitado'
        if row['Soma Atendimento Setorial'] == row['Soma Valor Solicitado']:
            return 1  # Retorna 1 se forem iguais
        else:
            return 2  # Retorna 2 se forem diferentes
    else:
        # Se a soma de 'Atendimento Setorial' for zero, verificar a presença da emenda na aba 'Bancada Impositiva - Consulta'
        if checa_bancada_impositiva(row['Sequencial da Emenda'], df_bancada):
            return 2  # Retorna 2 se a emenda for encontrada na 'Bancada Impositiva - Consulta'
        else:
            return 3  # Retorna 3 se a emenda não for encontrada na 'Bancada Impositiva - Consulta'

In [44]:
# Função para simular a fórmula de 'ID Uso'
def calcula_id_uso(row,df_criterio):
        
    # Contagem condicional em 'Critério IU 6'
    UO = int(row['UO'])
    count_x = df_criterio['UO'].eq(UO).sum()  # Equivalente a CONT.SE('Critério IU 6'!A:A;X2)
    last_4_of_y = row['Funcional'][-4:]  # Equivalente a DIREITA(Y2;4)
    count_y = df_criterio['Ação'].eq(last_4_of_y).sum()  # Equivalente a CONT.SE('Critério IU 6'!B:B;DIREITA(Y2;4))
    
    # Condicional para retornar 6 ou 0
    if count_x == 1 and count_y == 1:
        return 6
    else:
        return 0

In [45]:
def calcula_parecer(row):
    # Simula a lógica da fórmula =SE(AD2=1;1;SE(AD2=2;13;38)) onde AD2 é a coluna 'Decisão'
    if row['Decisão Parecer'] == 1:
        return 1
    elif row['Decisão Parecer'] == 2:
        return 13
    else:
        return 38

In [46]:
def formata_dataframe(df_relator):
    # Atualizar o dataframe original, mantendo apenas até a última linha com valor numérico.
    # Remove a última linha de contabilização e possíveis linhas vazias ao final da tabela
    ultima_linha_numerica = df_relator[df_relator['Emenda'].apply(lambda x: pd.to_numeric(x, errors='coerce')).notna()].index[-1]
    df_relator = df_relator.loc[:ultima_linha_numerica].copy()
    # Preencher células mescladas (NaN) com o último valor válido
    df_relator = df_relator.ffill()
    return df_relator

In [47]:
def somas_por_emendas(df_relator):
    
    somas_atendimento_setorial = df_relator.groupby('Emenda')['Atendimento Setorial'].sum()
    somas_valor_solicitado = df_relator.groupby('Emenda')['Valor Solicitado'].sum()

    # Adiciona essas somas como novas colunas ao dataframe original, usando o valor da emenda como referência
    df_relator = df_relator.merge(somas_atendimento_setorial.rename('Soma Atendimento Setorial'), 
                              left_on='Emenda', right_index=True, how='left')
    df_relator = df_relator.merge(somas_valor_solicitado.rename('Soma Valor Solicitado'), 
                              left_on='Emenda', right_index=True, how='left')
    return df_relator

In [48]:
def somas_parcelas_impositivas(df_bancada):

    somas_parcela_impositiva = df_bancada.groupby('Emenda')['Valor Solicitado'].sum()
    df_bancada = df_bancada.merge(somas_parcela_impositiva.rename('Soma Valor Solicitado'), 
                              left_on='Emenda', right_index=True, how='left')
    
    return df_bancada

In [49]:
def mapeia_parcelas_impositivas(df_bancada,df_relator):
    
    # Criar um dicionário de mapeamento a partir do DataFrame df_bancada
    mapeamento = dict(zip(df_bancada['Emenda'], df_bancada['Soma Valor Solicitado']))

    # Atualizar a coluna 'Tem parcela impositiva?' do df_relator usando o método .map()
    # Caso não haja correspondência, preenche com '-'
    df_relator['Tem parcela impositiva?'] = df_relator['Emenda'].map(mapeamento).fillna('-')

    return df_relator

In [50]:
    xw.Book("processaemendas.xlsm").set_mock_caller()
    # Conectar ao livro de trabalho ativo no xlwings
    wb = xw.Book.caller()  # Isso conecta ao arquivo Excel aberto pelo xlwings
    sheet_plansheet = wb.sheets['Sheet1']
    sheet_plansheet.range('A1').value = 'Gerando planilha Lexor...'
  

    # Carregar as abas como DataFrames
    sheet_apropriacao = wb.sheets['Relator - Coletivas Apropriação']
    sheet_criterio = wb.sheets['Critério IU 6']
    sheet_bancada = wb.sheets['Bancada Impositiva - Consulta']

    df_relator =  sheet_apropriacao.used_range.options(pd.DataFrame, index=False, header=True).value
    df_criterio = sheet_criterio.used_range.options(pd.DataFrame, index=False, header=True).value
    df_bancada = sheet_bancada.used_range.options(pd.DataFrame, index=False, header=True).value
    df_bancada = df_bancada.ffill()

 
   

In [52]:
df_relator

,Emenda,Emenda (Tipo),Autor,UO (Cod/Desc),Funcional (Sub),Ação + Subtítulo,Tem parcela impositiva?,GND,MA,RP,...,Valor,UO,Funcional,Subtítulo Padrão ou Não Padrão Existente,Subtítulo Novo,Localidade,Justificativa,Decisão Parecer,Parecer Padrão,Texto Parecer Específico
0,50030001,ACRÉSCIMO - APROPRIAÇÃO,COM. MEIO AMB DESENV SUSTENTÁVEL,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,17.511.2322.00VJ.0001,"APOIO À IMPLANTAÇÃO, AMPLIAÇÃO OU MELHORIA DE ...",'=SE(ÉERROS(PROCV(A2;'Bancada Impositiva - Con...,4,40,8,...,'=M2,56101,17.511.2322.00VJ,0001,None,None,/ Atendimento de emendas coletivas de apropri...,=SE(SOMASE(A:A;A2;M:M)<>0;SE(SOMASE(A:A;A2;M:M...,'=SE(AE2=1;1;SE(AE2=2;13;38)),None
1,50170003,INCLUSÃO - APROPRIAÇÃO,COM. FINANÇAS E TRIBUTAÇÃO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,28.846.2320.00CW.0001,SUBVENÇÃO ECONÔMICA DESTINADA À AMPLIAÇÃO DO A...,'=SE(ÉERROS(PROCV(A2;'Bancada Impositiva - Con...,3,90,8,...,'=M2,56101,28.846.2320.00CW,0001,None,None,/ Atendimento de emendas coletivas de apropri...,'=SE(SOMASE(A:A;A2;M:M)<>0;SE(SOMASE(A:A;A2;M:...,'=SE(AE2=1;1;SE(AE2=2;13;38)),None
2,50230002,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV. URBANO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.0001,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,'=SE(ÉERROS(PROCV(A2;'Bancada Impositiva - Con...,4,90,8,...,'=M2,56101,15.451.2319.00T1,0001,None,None,/ Atendimento de emendas coletivas de apropri...,'=SE(SOMASE(A:A;A2;M:M)<>0;SE(SOMASE(A:A;A2;M:...,'=SE(AE2=1;1;SE(AE2=2;13;38)),None
3,50230003,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV. URBANO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,17.512.2322.00TO.0001,"APOIO À IMPLANTAÇÃO, AMPLIAÇÃO OU MELHORIAS DE...",'=SE(ÉERROS(PROCV(A2;'Bancada Impositiva - Con...,4,90,8,...,'=M2,56101,17.512.2322.00TO,0001,None,None,/ Atendimento de emendas coletivas de apropri...,'=SE(SOMASE(A:A;A2;M:M)<>0;SE(SOMASE(A:A;A2;M:...,'=SE(AE2=1;1;SE(AE2=2;13;38)),None
4,50230004,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV. URBANO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,28.845.2320.00CY.0001,TRANSFERÊNCIAS AO FUNDO DE DESENVOLVIMENTO SOC...,'=SE(ÉERROS(PROCV(A2;'Bancada Impositiva - Con...,3,90,8,...,'=M2,56101,28.845.2320.00CY,0001,None,None,/ Atendimento de emendas coletivas de apropri...,'=SE(SOMASE(A:A;A2;M:M)<>0;SE(SOMASE(A:A;A2;M:...,'=SE(AE2=1;1;SE(AE2=2;13;38)),None
5,50310004,INCLUSÃO - APROPRIAÇÃO,COM. FISC FINANCEIRA E CONTROLE,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,17.512.2322.00TN.0001,"APOIO À IMPLANTAÇÃO, AMPLIAÇÃO OU MELHORIAS EM...",'=SE(ÉERROS(PROCV(A2;'Bancada Impositiva - Con...,4,99,6,...,'=M2,56101,17.512.2322.00TN,0001,None,None,/ Atendimento de emendas coletivas de apropri...,'=SE(SOMASE(A:A;A2;M:M)<>0;SE(SOMASE(A:A;A2;M:...,'=SE(AE2=1;1;SE(AE2=2;13;38)),None
6,60110001,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV REGIONAL E TURISMO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.5601.00SY.0001,"APOIO A PROJETOS E OBRAS DE REABILITAÇÃO, DE A...",'=SE(ÉERROS(PROCV(A2;'Bancada Impositiva - Con...,4,30,8,...,'=M2,56101,15.451.5601.00SY,0001,None,None,/ Atendimento de emendas coletivas de apropri...,'=SE(SOMASE(A:A;A2;M:M)<>0;SE(SOMASE(A:A;A2;M:...,'=SE(AE2=1;1;SE(AE2=2;13;38)),None
7,None,None,None,None,None,None,'=SE(ÉERROS(PROCV(A2;'Bancada Impositiva - Con...,4,40,8,...,'=M2,56101,15.451.5601.00SY,0001,None,None,/ Atendimento de emendas coletivas de apropri...,'=SE(SOMASE(A:A;A2;M:M)<>0;SE(SOMASE(A:A;A2;M:...,'=SE(AE2=1;1;SE(AE2=2;13;38)),None
8,60110002,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV REGIONAL E TURISMO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.0001,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,'=SE(ÉERROS(PROCV(A2;'Bancada Impositiva - Con...,4,30,8,...,'=M2,56101,15.451.2319.00T1,0001,None,None,/ Atendimento de emendas coletivas de apropri...,'=SE(SOMASE(A:A;A2;M:M)<>0;SE(SOMASE(A:A;A2;M:...,'=SE(AE2=1;1;SE(AE2=2;13;38)),None
9,None,None,None,None,None,None,'=SE(ÉERROS(PROCV(A2;'Bancada Impositiva - Con...

In [53]:
df_bancada = somas_parcelas_impositivas(df_bancada)
df_relator = formata_dataframe(df_relator)

In [55]:
df_relator

,Emenda,Emenda (Tipo),Autor,UO (Cod/Desc),Funcional (Sub),Ação + Subtítulo,Tem parcela impositiva?,GND,MA,RP,...,Valor,UO,Funcional,Subtítulo Padrão ou Não Padrão Existente,Subtítulo Novo,Localidade,Justificativa,Decisão Parecer,Parecer Padrão,Texto Parecer Específico
0,50030001,ACRÉSCIMO - APROPRIAÇÃO,COM. MEIO AMB DESENV SUSTENTÁVEL,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,17.511.2322.00VJ.0001,"APOIO À IMPLANTAÇÃO, AMPLIAÇÃO OU MELHORIA DE ...",'=SE(ÉERROS(PROCV(A2;'Bancada Impositiva - Con...,4,40,8,...,'=M2,56101,17.511.2322.00VJ,0001,None,None,/ Atendimento de emendas coletivas de apropri...,=SE(SOMASE(A:A;A2;M:M)<>0;SE(SOMASE(A:A;A2;M:M...,'=SE(AE2=1;1;SE(AE2=2;13;38)),None
1,50170003,INCLUSÃO - APROPRIAÇÃO,COM. FINANÇAS E TRIBUTAÇÃO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,28.846.2320.00CW.0001,SUBVENÇÃO ECONÔMICA DESTINADA À AMPLIAÇÃO DO A...,'=SE(ÉERROS(PROCV(A2;'Bancada Impositiva - Con...,3,90,8,...,'=M2,56101,28.846.2320.00CW,0001,None,None,/ Atendimento de emendas coletivas de apropri...,'=SE(SOMASE(A:A;A2;M:M)<>0;SE(SOMASE(A:A;A2;M:...,'=SE(AE2=1;1;SE(AE2=2;13;38)),None
2,50230002,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV. URBANO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.0001,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,'=SE(ÉERROS(PROCV(A2;'Bancada Impositiva - Con...,4,90,8,...,'=M2,56101,15.451.2319.00T1,0001,None,None,/ Atendimento de emendas coletivas de apropri...,'=SE(SOMASE(A:A;A2;M:M)<>0;SE(SOMASE(A:A;A2;M:...,'=SE(AE2=1;1;SE(AE2=2;13;38)),None
3,50230003,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV. URBANO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,17.512.2322.00TO.0001,"APOIO À IMPLANTAÇÃO, AMPLIAÇÃO OU MELHORIAS DE...",'=SE(ÉERROS(PROCV(A2;'Bancada Impositiva - Con...,4,90,8,...,'=M2,56101,17.512.2322.00TO,0001,None,None,/ Atendimento de emendas coletivas de apropri...,'=SE(SOMASE(A:A;A2;M:M)<>0;SE(SOMASE(A:A;A2;M:...,'=SE(AE2=1;1;SE(AE2=2;13;38)),None
4,50230004,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV. URBANO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,28.845.2320.00CY.0001,TRANSFERÊNCIAS AO FUNDO DE DESENVOLVIMENTO SOC...,'=SE(ÉERROS(PROCV(A2;'Bancada Impositiva - Con...,3,90,8,...,'=M2,56101,28.845.2320.00CY,0001,None,None,/ Atendimento de emendas coletivas de apropri...,'=SE(SOMASE(A:A;A2;M:M)<>0;SE(SOMASE(A:A;A2;M:...,'=SE(AE2=1;1;SE(AE2=2;13;38)),None
5,50310004,INCLUSÃO - APROPRIAÇÃO,COM. FISC FINANCEIRA E CONTROLE,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,17.512.2322.00TN.0001,"APOIO À IMPLANTAÇÃO, AMPLIAÇÃO OU MELHORIAS EM...",'=SE(ÉERROS(PROCV(A2;'Bancada Impositiva - Con...,4,99,6,...,'=M2,56101,17.512.2322.00TN,0001,None,None,/ Atendimento de emendas coletivas de apropri...,'=SE(SOMASE(A:A;A2;M:M)<>0;SE(SOMASE(A:A;A2;M:...,'=SE(AE2=1;1;SE(AE2=2;13;38)),None
6,60110001,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV REGIONAL E TURISMO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.5601.00SY.0001,"APOIO A PROJETOS E OBRAS DE REABILITAÇÃO, DE A...",'=SE(ÉERROS(PROCV(A2;'Bancada Impositiva - Con...,4,30,8,...,'=M2,56101,15.451.5601.00SY,0001,None,None,/ Atendimento de emendas coletivas de apropri...,'=SE(SOMASE(A:A;A2;M:M)<>0;SE(SOMASE(A:A;A2;M:...,'=SE(AE2=1;1;SE(AE2=2;13;38)),None
7,60110001,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV REGIONAL E TURISMO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.5601.00SY.0001,"APOIO A PROJETOS E OBRAS DE REABILITAÇÃO, DE A...",'=SE(ÉERROS(PROCV(A2;'Bancada Impositiva - Con...,4,40,8,...,'=M2,56101,15.451.5601.00SY,0001,None,None,/ Atendimento de emendas coletivas de apropri...,'=SE(SOMASE(A:A;A2;M:M)<>0;SE(SOMASE(A:A;A2;M:...,'=SE(AE2=1;1;SE(AE2=2;13;38)),None
8,60110002,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV REGIONAL E TURISMO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.0001,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,'=SE(ÉERROS(PROCV(A2;'Bancada Impositiva - Con...,4,30,8,...,'=M2,56101,15.451.2319.00T1,0001,None,None,/ Atendimento de emendas coletivas de apropri...,'=SE(SOM

In [56]:
df_relator = somas_por_emendas(df_relator)
df_relator = mapeia_parcelas_impositivas(df_bancada,df_relator)


In [59]:

df_bancada

,Emenda,Emenda (Tipo),Autor,UO (Cod/Desc),Funcional,Ação + Subtítulo,GND,MA,RP,IU,...,UO,Funcional,Subtítulo Padrão ou Não Padrão Existente,Subtítulo Novo,Localidade,Justificativa,Decisão Parecer,Parecer Padrão,Texto Parecer Específico,Soma Valor Solicitado
0,71020009,INCLUSÃO - APROPRIAÇÃO,BANCADA DO ACRE,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.0172,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,4,30,7,0,...,56101,15.451.2319.00T1,0172,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None,1000000.0
1,71050003,INCLUSÃO - APROPRIAÇÃO,BANCADA DO AMAPA,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.5601.00SY.7003,"APOIO A PROJETOS E OBRAS DE REABILITAÇÃO, DE A...",4,30,7,0,...,56101,15.451.5601.00SY,7003,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None,10000000.0
2,71050004,INCLUSÃO - APROPRIAÇÃO,BANCADA DO AMAPA,56902 - FUNDO NACIONAL DE HABITAÇÃO DE INTERES...,16.482.2320.00TI.7001,APOIO À PRODUÇÃO HABITACIONAL DE INTERESSE SOC...,4,30,7,0,...,56902,16.482.2320.00TI,7001,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None,15624188.0
3,71090007,INCLUSÃO - APROPRIAÇÃO,BANCADA DO ESPIRITO SANTO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.7012,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,4,30,7,0,...,56101,15.451.2319.00T1,7012,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None,25000000.0
4,71100003,ACRÉSCIMO - APROPRIAÇÃO,BANCADA DE GOIAS,56202 - COMPANHIA BRASILEIRA DE TRENS URBANOS ...,15.453.0032.2843.0001,FUNCIONAMENTO DOS SISTEMAS DE TRANSPORTE FERRO...,4,90,7,0,...,56202,15.453.0032.2843,0001,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None,8446651.0
5,71100014,INCLUSÃO - APROPRIAÇÃO,BANCADA DE GOIAS,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,28.846.2320.00CW.0052,SUBVENÇÃO ECONÔMICA DESTINADA À AMPLIAÇÃO DO A...,3,90,7,0,...,56101,28.846.2320.00CW,0052,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None,10000000.0
6,71120008,INCLUSÃO - APROPRIAÇÃO,BANCADA DO MATO GROSSO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.5401,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,4,40,7,0,...,56101,15.451.2319.00T1,5401,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None,28812094.0
7,71120009,INCLUSÃO - APROPRIAÇÃO,BANCADA DO MATO GROSSO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.127.5602.00SW.5411,APOIO À REGULARIZAÇÃO FUNDIÁRIA URBANA - NO MU...,4,40,7,0,...,56101,15.127.5602.00SW,5411,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None,14000000.0
8,71130010,INCLUSÃO - APROPRIAÇÃO,BANCADA DO MATO GROSSO DO SUL,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,28.846.2320.00CW.0054,SUBVENÇÃO ECONÔMICA DESTINADA À AMPLIAÇÃO DO A...,3,90,7,0,...,56101,28.846.2320.00CW,0054,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None,35000000.0
9,71160012,INCLUSÃO - APROPRIAÇÃO,BANCADA DA PARAIBA,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.1436,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,4,40,7,0,...,56101,15.451.2319.00T1,1436,None,None,/ Atendimento de emendas coletivas de apropri...,''=SE(ÉERROS(PROCV(O2;'Relator - Coletivas Apr...,'=SE(AD2=1;1;SE(AD2=2;13;38)),None,10300000.0


In [60]:
    # Aplicando a função para calcular 'ID Uso'
    df_relator['ID Uso'] = df_relator.apply(lambda row: calcula_id_uso(row,df_criterio), axis=1)
    # Aplicando a função para calcular 'Decisão Parecer'
    df_relator['Decisão Parecer'] = df_relator.apply(lambda row: calcula_decisao(row, df_bancada), axis=1)
    # Aplicando a função para calcular 'Parecer'
    df_relator['Parecer Padrão'] = df_relator.apply(lambda row: calcula_parecer(row), axis=1)
    
    
   

In [61]:
df_relator

,Emenda,Emenda (Tipo),Autor,UO (Cod/Desc),Funcional (Sub),Ação + Subtítulo,Tem parcela impositiva?,GND,MA,RP,...,Funcional,Subtítulo Padrão ou Não Padrão Existente,Subtítulo Novo,Localidade,Justificativa,Decisão Parecer,Parecer Padrão,Texto Parecer Específico,Soma Atendimento Setorial,Soma Valor Solicitado
0,50030001,ACRÉSCIMO - APROPRIAÇÃO,COM. MEIO AMB DESENV SUSTENTÁVEL,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,17.511.2322.00VJ.0001,"APOIO À IMPLANTAÇÃO, AMPLIAÇÃO OU MELHORIA DE ...",-,4,40,8,...,17.511.2322.00VJ,0001,None,None,/ Atendimento de emendas coletivas de apropri...,3,38,None,0,2.002000e+08
1,50170003,INCLUSÃO - APROPRIAÇÃO,COM. FINANÇAS E TRIBUTAÇÃO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,28.846.2320.00CW.0001,SUBVENÇÃO ECONÔMICA DESTINADA À AMPLIAÇÃO DO A...,-,3,90,8,...,28.846.2320.00CW,0001,None,None,/ Atendimento de emendas coletivas de apropri...,3,38,None,0,4.000000e+05
2,50230002,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV. URBANO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.0001,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,-,4,90,8,...,15.451.2319.00T1,0001,None,None,/ Atendimento de emendas coletivas de apropri...,3,38,None,0,1.065000e+09
3,50230003,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV. URBANO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,17.512.2322.00TO.0001,"APOIO À IMPLANTAÇÃO, AMPLIAÇÃO OU MELHORIAS DE...",-,4,90,8,...,17.512.2322.00TO,0001,None,None,/ Atendimento de emendas coletivas de apropri...,3,38,None,0,5.000000e+04
4,50230004,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV. URBANO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,28.845.2320.00CY.0001,TRANSFERÊNCIAS AO FUNDO DE DESENVOLVIMENTO SOC...,-,3,90,8,...,28.845.2320.00CY,0001,None,None,/ Atendimento de emendas coletivas de apropri...,3,38,None,0,2.005000e+07
5,50310004,INCLUSÃO - APROPRIAÇÃO,COM. FISC FINANCEIRA E CONTROLE,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,17.512.2322.00TN.0001,"APOIO À IMPLANTAÇÃO, AMPLIAÇÃO OU MELHORIAS EM...",-,4,99,6,...,17.512.2322.00TN,0001,None,None,/ Atendimento de emendas coletivas de apropri...,3,38,None,0,0.000000e+00
6,60110001,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV REGIONAL E TURISMO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.5601.00SY.0001,"APOIO A PROJETOS E OBRAS DE REABILITAÇÃO, DE A...",-,4,30,8,...,15.451.5601.00SY,0001,None,None,/ Atendimento de emendas coletivas de apropri...,3,38,None,0,3.850000e+08
7,60110001,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV REGIONAL E TURISMO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.5601.00SY.0001,"APOIO A PROJETOS E OBRAS DE REABILITAÇÃO, DE A...",-,4,40,8,...,15.451.5601.00SY,0001,None,None,/ Atendimento de emendas coletivas de apropri...,3,38,None,0,3.850000e+08
8,60110002,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV REGIONAL E TURISMO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.0001,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,-,4,30,8,...,15.451.2319.00T1,0001,None,None,/ Atendimento de emendas coletivas de apropri...,3,38,None,0,9.550000e+08
9,60110002,ACRÉSCIMO - APROPRIAÇÃO,COM. DESENV REGIONAL E TURISMO,56101 - MINISTÉRIO DAS CIDADES - ADMINISTRAÇÃO...,15.451.2319.00T1.0001,APOIO À POLÍTICA NACIONAL DE DESENVOLVIMENTO U...,-,4,40,8,...,15.451.2319.00T1,0001,None,None,/ Atendimento de emendas coletivas de apropri...,3,38,None,0,9.550000e+08


In [ ]:
    # Escrever o DataFrame `df_lexor` de volta a partir da coluna "Sequencial da Emenda"
    sheet_apropriacao.range('A1').options(index=False).value = df_relator.iloc[:, :-2] # 'reset_index' evita o índice do Dataframe. Descartando as duas últimas colunas de soma criadas anteriormente

    sheet_plansheet.range('A1').value = 'Planilha Lexor gerada com sucesso!'

In [ ]:
if __name__ == "__main__":
    xw.Book("processaemendas.xlsm").set_mock_caller()
    main()